## Data : https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers

In [1]:
%matplotlib inline 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# Load Data 
data = pd.read_csv("BankChurners.csv").drop(['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 
                                             'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], 
                                            axis=1).drop_duplicates()
data.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [3]:
data.shape

(10127, 20)

## 결측치 확인

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

In [5]:
data.shape

(10127, 20)

Customer의 나이를 예측해보는 걸로

In [6]:
x = data[['Contacts_Count_12_mon','Months_Inactive_12_mon','Total_Amt_Chng_Q4_Q1','Total_Relationship_Count','Total_Trans_Amt','Avg_Utilization_Ratio','Total_Revolving_Bal','Total_Ct_Chng_Q4_Q1','Total_Trans_Ct']]
y = data['Customer_Age']

In [7]:
numeric_columns = data.select_dtypes(include=['int64']).columns.to_list()
object_columns = data.select_dtypes(include=['object']).columns.to_list()

In [8]:
x = pd.concat([data[numeric_columns], pd.get_dummies(data[object_columns])], axis=1)

## Data Split

In [9]:
x.shape, y.shape

((10127, 34), (10127,))

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state=2020)

In [12]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((8101, 34), (2026, 34), (8101,), (2026,))

## Modeling

In [13]:
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

### XGBRegressor

In [14]:
xgb = XGBRegressor()

In [15]:
xgb.fit(train_x, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [16]:
pred_xgb = xgb.predict(test_x)

In [17]:
mean_squared_error(test_y, pred_xgb)

1.6707652901195408e-08

In [24]:
pred_xgb

array([36.999958, 26.999783, 49.000057, ..., 60.000168, 43.999844,
       41.99987 ], dtype=float32)

### RandomForestRegressor

In [18]:
rf = RandomForestRegressor()

In [19]:
rf.fit(train_x, train_y)

RandomForestRegressor()

In [20]:
pred_rf = rf.predict(test_x)

In [21]:
mean_squared_error(test_y, pred_rf)

0.0

In [22]:
pred_rf

array([37., 27., 49., ..., 60., 44., 42.])

In [23]:
test_y

7025    37
1513    27
43      49
5522    51
2087    37
        ..
9218    34
9212    47
938     60
15      44
6316    42
Name: Customer_Age, Length: 2026, dtype: int64